# Data to df

In [1]:
import requests
import bs4
import pandas as pd
import re
import numpy as np

res = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M')
soup = bs4.BeautifulSoup(res.text, 'html.parser')

h3 = soup.select('tr')
rows=[]

for i in h3:
    rows.append(i.text)

k1 = []

for i in range(len(h3)):
    k = h3[i].text
    k1.append(re.findall(r"[\w ']+", k))

df = pd.DataFrame(data = k1)

df.columns = df.iloc[0]

df.drop(df.index[0],inplace=True)

df.drop(df.index[180:],inplace=True)


# Deleting Not assigned from Borough

In [2]:
indexnames = df[df['Borough']=='Not assigned'].index

df.drop(indexnames,inplace=True)

# Combining Neighbourhoods

In [3]:
df.astype('str')

df = df.replace({None:''})

df['Neighborhood'] = df.iloc[:,2:].apply(lambda x: ','.join(x), axis = 1)

df.head()

,Postal Code,Borough,Neighborhood,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,M3A,North York,"Parkwoods,,,,,,,,,,,,,,,,,,,,,,,,,,,,",,,,,,,,...,,,,,,,,,,
4,M4A,North York,"Victoria Village,,,,,,,,,,,,,,,,,,,,,,,,,,,,",,,,,,,,...,,,,,,,,,,
5,M5A,Downtown Toronto,"Regent Park, Harbourfront,,,,,,,,,,,,,,,,,,,,,...",Harbourfront,,,,,,,...,,,,,,,,,,
6,M6A,North York,"Lawrence Manor, Lawrence Heights,,,,,,,,,,,,,,...",Lawrence Heights,,,,,,,...,,,,,,,,,,
7,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government,,,...",Ontario Provincial Government,,,,,,,...,,,,,,,,,,


# Deleting Remaining Columns

In [4]:
df.drop(columns=df.iloc[:,3:],axis=1,inplace=True)

df.head()

,Postal Code,Borough,Neighborhood
3,M3A,North York,"Parkwoods,,,,,,,,,,,,,,,,,,,,,,,,,,,,"
4,M4A,North York,"Victoria Village,,,,,,,,,,,,,,,,,,,,,,,,,,,,"
5,M5A,Downtown Toronto,"Regent Park, Harbourfront,,,,,,,,,,,,,,,,,,,,,..."
6,M6A,North York,"Lawrence Manor, Lawrence Heights,,,,,,,,,,,,,,..."
7,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government,,,..."


# Deleting extra terms in neighborhood

In [5]:
df['Neighborhood'] = df['Neighborhood'].str.split(',,').str[0]

df.head()

,Postal Code,Borough,Neighborhood
3,M3A,North York,Parkwoods
4,M4A,North York,Victoria Village
5,M5A,Downtown Toronto,"Regent Park, Harbourfront"
6,M6A,North York,"Lawrence Manor, Lawrence Heights"
7,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


In [6]:
df.reset_index(drop=True)

,Postal Code,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
...,...,...,...
98,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North"
99,M4Y,Downtown Toronto,Church and Wellesley
100,M7Y,East Toronto,"Business reply mail Processing Centre, South C..."
101,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu..."


# Shape

In [7]:
df.shape

(103, 3)

# Adding Coordinates using CSV

In [8]:
cord = pd.read_csv("Geospatial_Coordinates.csv")


df2 = pd.merge(df,cord,on='Postal Code')

df2.head()

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494


In [9]:
df2.shape

(103, 5)

# Boroughs and Neighborhood

In [10]:
print('The dataframe has {} boroughs and {} neighborhoods.'.format(
        len(df2['Borough'].unique()),
        df2.shape[0]
    )
)

The dataframe has 10 boroughs and 103 neighborhoods.


In [11]:
from geopy.geocoders import Nominatim 

In [12]:
import folium

# Display all data in a map

In [13]:
address = 'Toronto, CAN'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.8184944, -79.3309468.


In [14]:
# create map of New York using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=20)

# add markers to map
for lat, lng, borough, neighborhood in zip(df2['Latitude'], df2['Longitude'], df2['Borough'], df2['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

# dataframe with only borough near toronto

In [15]:
tor = [col for col in df2['Borough'] if 'Toronto' in col]
tor = np.unique(tor)

tor

array(['Central Toronto', 'Downtown Toronto', 'East Toronto',
       'West Toronto'], dtype='<U16')

In [16]:
df3 = df2[df2.Borough.isin(tor)].reset_index(drop=True)

df3.head()

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
1,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
2,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937
3,M5C,Downtown Toronto,"St, James Town",43.651494,-79.375418
4,M4E,East Toronto,The Beaches,43.676357,-79.293031


In [17]:
df3.shape

(39, 5)

# Map with Borough that contain the word Toronto

In [18]:
# create map of New York using latitude and longitude values
map_toronto1 = folium.Map(location=[latitude, longitude], zoom_start=20)

# add markers to map
for lat, lng, borough, neighborhood in zip(df3['Latitude'], df3['Longitude'], df3['Borough'], df3['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto1)  
    
map_toronto1

# Define Foursquare Credentials

In [19]:
CLIENT_ID = 'LTTXUALGOYNOHT1FAUYAMJLR022UCMYOX40USFX1ZYBQT' # your Foursquare ID
CLIENT_SECRET = '3LG5U52HMBZ2SLQML0FJG3HKQLF0UPLA4GWSKMJWKI1G' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: LTTXUALGOYNOHT1FAUYAMJLR022UCMYOX40USFX1ZYBQT
CLIENT_SECRET:3LG5U52HMBZ2SLQML0FJG3HKQLF0UPLA4GWSKMJWKI1G


In [20]:
df3.loc[0, 'Neighborhood']

'Regent Park, Harbourfront'

In [21]:
neighborhood_latitude = df3.loc[0, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = df3.loc[0, 'Longitude'] # neighborhood longitude value

neighborhood_name = df3.loc[0, 'Neighborhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of Regent Park, Harbourfront are 43.6542599, -79.3606359.


In [22]:
radius = 500
limit = 30
#//latit = str(neighborhood.latitude)
#//longi = str(neighborhood.longitude)
url = 'https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&ll={},{}&v={}&radius={}&limit={}'.format(CLIENT_ID,CLIENT_SECRET,neighborhood_latitude,neighborhood_longitude,VERSION,radius,limit)
url

'https://api.foursquare.com/v2/venues/explore?client_id=LTTXUALGOYNOHT1FAUYAMJLR022UCMYOX40USFX1ZYBQTYTZ&client_secret=3LG5U52HMBZ2SLQML0FJG3HKQLF0UPLA4GWSKMJWKI1GV1VI&ll=43.6542599,-79.3606359&v=20180605&radius=500&limit=30'

In [23]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5ef55f514cb2e80ec637bc00'},
 'response': {'suggestedFilters': {'header': 'Tap to show:',
   'filters': [{'name': 'Open now', 'key': 'openNow'}]},
  'headerLocation': 'Corktown',
  'headerFullLocation': 'Corktown, Toronto',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 44,
  'suggestedBounds': {'ne': {'lat': 43.6587599045, 'lng': -79.3544279001486},
   'sw': {'lat': 43.6497598955, 'lng': -79.36684389985142}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '54ea41ad498e9a11e9e13308',
       'name': 'Roselle Desserts',
       'location': {'address': '362 King St E',
        'crossStreet': 'Trinity St',
        'lat': 43.653446723052674,
        'lng': -79.3620167174383,
        'labeledLatLngs': [{'label': 'display',
 

In [24]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [26]:
#import pandas as pd
venues = results['response']['groups'][0]['items']
    
nearby_venues = pd.json_normalize(venues) # flatten JSON

# filter columns  
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng
0,Roselle Desserts,Bakery,43.653447,-79.362017
1,Tandem Coffee,Coffee Shop,43.653559,-79.361809
2,Cooper Koo Family YMCA,Distribution Center,43.653249,-79.358008
3,Body Blitz Spa East,Spa,43.654735,-79.359874
4,Dominion Pub and Kitchen,Pub,43.656919,-79.358967


In [35]:
nearby_venues

,name,categories,lat,lng
0,Roselle Desserts,Bakery,43.653447,-79.362017
1,Tandem Coffee,Coffee Shop,43.653559,-79.361809
2,Cooper Koo Family YMCA,Distribution Center,43.653249,-79.358008
3,Body Blitz Spa East,Spa,43.654735,-79.359874
4,Dominion Pub and Kitchen,Pub,43.656919,-79.358967
5,Corktown Common,Park,43.655618,-79.356211
6,Impact Kitchen,Restaurant,43.656369,-79.356980
7,Morning Glory Cafe,Breakfast Spot,43.653947,-79.361149
8,The Extension Room,Gym / Fitness Center,43.653313,-79.359725
9,The Distillery Historic District,Historic Site,43.650244,-79.359323


In [36]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

30 venues were returned by Foursquare.


In [37]:
LIMIT =10

# Function for nearby venues

In [38]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)                         
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

# Nearby Venues

In [39]:
toronto_venues = getNearbyVenues(names=df3['Neighborhood'],
                                   latitudes=df3['Latitude'],
                                   longitudes=df3['Longitude']
                                  )

Regent Park, Harbourfront
Queen's Park, Ontario Provincial Government
Garden District, Ryerson
St, James Town
The Beaches
Berczy Park
Central Bay Street
Christie
Richmond, Adelaide, King
Dufferin, Dovercourt Village
Harbourfront East, Union Station, Toronto Islands
Little Portugal, Trinity
The Danforth West, Riverdale
Toronto Dominion Centre, Design Exchange
Brockton, Parkdale Village, Exhibition Place
India Bazaar, The Beaches West
Commerce Court, Victoria Hotel
Studio District
Lawrence Park
Roselawn
Davisville North
Forest Hill North , West, Forest Hill Road Park
High Park, The Junction South
North Toronto West,  Lawrence Park
The Annex, North Midtown, Yorkville
Parkdale, Roncesvalles
Davisville
University of Toronto, Harbord
Runnymede, Swansea
Moore Park, Summerhill East
Kensington Market, Chinatown, Grange Park
Summerhill West, Rathnelly, South Hill, Forest Hill SE, Deer Park
CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport


In [41]:
print(toronto_venues.shape)
toronto_venues.head()

(352, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Regent Park, Harbourfront",43.65426,-79.360636,Roselle Desserts,43.653447,-79.362017,Bakery
1,"Regent Park, Harbourfront",43.65426,-79.360636,Tandem Coffee,43.653559,-79.361809,Coffee Shop
2,"Regent Park, Harbourfront",43.65426,-79.360636,Cooper Koo Family YMCA,43.653249,-79.358008,Distribution Center
3,"Regent Park, Harbourfront",43.65426,-79.360636,Body Blitz Spa East,43.654735,-79.359874,Spa
4,"Regent Park, Harbourfront",43.65426,-79.360636,Dominion Pub and Kitchen,43.656919,-79.358967,Pub


In [43]:
toronto_venues['Venue Category']

0                    Bakery
1               Coffee Shop
2       Distribution Center
3                       Spa
4                       Pub
               ...         
347    Fast Food Restaurant
348              Skate Park
349              Restaurant
350           Garden Center
351           Auto Workshop
Name: Venue Category, Length: 352, dtype: object

# Grouping Neighborhoods

In [44]:
toronto_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Berczy Park,10,10,10,10,10,10
"Brockton, Parkdale Village, Exhibition Place",10,10,10,10,10,10
"Business reply mail Processing Centre, South Central Letter Processing Plant Toronto",10,10,10,10,10,10
"CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport",10,10,10,10,10,10
Central Bay Street,10,10,10,10,10,10
Christie,10,10,10,10,10,10
Church and Wellesley,10,10,10,10,10,10
"Commerce Court, Victoria Hotel",10,10,10,10,10,10
Davisville,10,10,10,10,10,10


In [46]:
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))

There are 122 uniques categories.


# Analyze each Neighborhood

In [47]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

,Yoga Studio,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Art Gallery,...,Steakhouse,Supermarket,Sushi Restaurant,Swim School,Tea Room,Thai Restaurant,Theme Restaurant,Trail,Vegetarian / Vegan Restaurant,Wine Bar
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [48]:
toronto_onehot.shape

(352, 122)

In [49]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped

,Neighborhood,Yoga Studio,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,...,Steakhouse,Supermarket,Sushi Restaurant,Swim School,Tea Room,Thai Restaurant,Theme Restaurant,Trail,Vegetarian / Vegan Restaurant,Wine Bar
0,Berczy Park,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.00,0.0,0.0,0.0,0.000000,0.1,0.0
1,"Brockton, Parkdale Village, Exhibition Place",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.00,0.0,0.0,0.0,0.000000,0.0,0.0
2,"Business reply mail Processing Centre, South C...",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.00,0.0,0.0,0.0,0.000000,0.0,0.0
3,"CN Tower, King and Spadina, Railway Lands, Har...",0.0,0.1,0.1,0.1,0.2,0.1,0.1,0.0,0.0,...,0.0,0.0,0.000000,0.00,0.0,0.0,0.0,0.000000,0.0,0.0
4,Central Bay Street,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.00,0.0,0.0,0.0,0.000000,0.0,0.0
5,Christie,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.00,0.0,0.0,0.0,0.000000,0.0,0.0
6,Church and Wellesley,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.00,0.0,0.0,0.1,0.000000,0.0,0.0
7,"Commerce Court, Victoria Hotel",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.00,0.1,0.0,0.0,0.000000,0.0,0.0
8,Davisville,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.100000,0.00,0.0,0.0,0.0,0.000000,0.0,0.0
9,Davisville North,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.00,0.0,0.0,0.0,0.000000,0.0,0.0


In [50]:
toronto_grouped.shape

(39, 122)

# Top 5 Venues

In [51]:
num_top_venues = 5

for hood in toronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Berczy Park----
               venue  freq
0       Cocktail Bar   0.1
1  French Restaurant   0.1
2             Museum   0.1
3       Liquor Store   0.1
4         Restaurant   0.1


----Brockton, Parkdale Village, Exhibition Place----
                venue  freq
0         Coffee Shop   0.2
1                Café   0.1
2  Italian Restaurant   0.1
3                 Gym   0.1
4           Pet Store   0.1


----Business reply mail Processing Centre, South Central Letter Processing Plant Toronto----
                  venue  freq
0            Skate Park   0.1
1  Fast Food Restaurant   0.1
2               Brewery   0.1
3            Restaurant   0.1
4         Burrito Place   0.1


----CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport----
              venue  freq
0    Airport Lounge   0.2
1             Plane   0.1
2  Airport Terminal   0.1
3          Boutique   0.1
4           Airport   0.1


----Central Bay Street----
                  

4           Bookstore   0.1




# Function for displaying top 10 venues

In [52]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [54]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Berczy Park,Museum,Concert Hall,Restaurant,Breakfast Spot,Farmers Market,Liquor Store,Park,French Restaurant,Vegetarian / Vegan Restaurant,Cocktail Bar
1,"Brockton, Parkdale Village, Exhibition Place",Coffee Shop,Furniture / Home Store,Gym,Bakery,Breakfast Spot,Italian Restaurant,Bar,Café,Pet Store,Creperie
2,"Business reply mail Processing Centre, South C...",Garden Center,Pizza Place,Skate Park,Auto Workshop,Farmers Market,Fast Food Restaurant,Brewery,Restaurant,Burrito Place,Comic Shop
3,"CN Tower, King and Spadina, Railway Lands, Har...",Airport Lounge,Boutique,Airport,Airport Food Court,Airport Gate,Airport Service,Airport Terminal,Plane,Harbor / Marina,Dessert Shop
4,Central Bay Street,Coffee Shop,Modern European Restaurant,Middle Eastern Restaurant,Bubble Tea Shop,Japanese Restaurant,Spa,Italian Restaurant,Dessert Shop,Comic Shop,Concert Hall


# Clustering

In [56]:
from sklearn.cluster import KMeans

In [57]:
# set number of clusters
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([2, 1, 2, 2, 0, 1, 2, 1, 1, 2])

In [73]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged = df3

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

toronto_merged.head() # check the last columns!

,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636,2,Distribution Center,Gym / Fitness Center,Bakery,Pub,Historic Site,Spa,Breakfast Spot,Restaurant,Park,Coffee Shop
1,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494,1,Sushi Restaurant,Coffee Shop,Yoga Studio,Park,Creperie,Beer Bar,Italian Restaurant,Distribution Center,Food & Drink Shop,Flea Market
2,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937,2,Clothing Store,Burrito Place,Ramen Restaurant,Plaza,Pizza Place,Comic Shop,Tea Room,Thai Restaurant,Café,Music Venue
3,Downtown Toronto,"St, James Town",43.651494,-79.375418,0,Coffee Shop,Cosmetics Shop,Gym,Japanese Restaurant,Restaurant,Creperie,Italian Restaurant,Middle Eastern Restaurant,Food Truck,Convenience Store
4,East Toronto,The Beaches,43.676357,-79.293031,2,Trail,Asian Restaurant,Health Food Store,Pub,Wine Bar,Cuban Restaurant,Dessert Shop,Department Store,Dance Studio,Creperie


In [75]:
import matplotlib.cm as cm
import matplotlib.colors as colors

# Display in a map

In [76]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

# Cluster 1

In [77]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
3,"St, James Town",Coffee Shop,Cosmetics Shop,Gym,Japanese Restaurant,Restaurant,Creperie,Italian Restaurant,Middle Eastern Restaurant,Food Truck,Convenience Store
6,Central Bay Street,Coffee Shop,Modern European Restaurant,Middle Eastern Restaurant,Bubble Tea Shop,Japanese Restaurant,Spa,Italian Restaurant,Dessert Shop,Comic Shop,Concert Hall


# Cluster2

In [78]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,"Queen's Park, Ontario Provincial Government",Sushi Restaurant,Coffee Shop,Yoga Studio,Park,Creperie,Beer Bar,Italian Restaurant,Distribution Center,Food & Drink Shop,Flea Market
7,Christie,Café,Grocery Store,Coffee Shop,Italian Restaurant,Candy Store,Restaurant,Diner,Comic Shop,Concert Hall,Convenience Store
9,"Dufferin, Dovercourt Village",Bakery,Bar,Middle Eastern Restaurant,Café,Pharmacy,Brewery,Bank,Music Venue,Grocery Store,Flea Market
12,"The Danforth West, Riverdale",Greek Restaurant,Ice Cream Shop,Cosmetics Shop,Brewery,Italian Restaurant,Yoga Studio,Fruit & Vegetable Store,Fountain,Diner,Comic Shop
13,"Toronto Dominion Centre, Design Exchange",Hotel,Gym,Café,Pub,Restaurant,Beer Bar,Bakery,Gym / Fitness Center,Coffee Shop,Tea Room
14,"Brockton, Parkdale Village, Exhibition Place",Coffee Shop,Furniture / Home Store,Gym,Bakery,Breakfast Spot,Italian Restaurant,Bar,Café,Pet Store,Creperie
16,"Commerce Court, Victoria Hotel",Café,Museum,Coffee Shop,Tea Room,Bakery,Pub,Restaurant,Gym,Gym / Fitness Center,Wine Bar
17,Studio District,Ice Cream Shop,Fish Market,Café,Pet Store,Bookstore,Sandwich Place,Bakery,Coffee Shop,Gay Bar,Flea Market
24,"The Annex, North Midtown, Yorkville",Café,Indian Restaurant,Vegetarian / Vegan Restaurant,Park,Middle Eastern Restaurant,Coffee Shop,History Museum,BBQ Joint,Burger Joint,Dance Studio
25,"Parkdale, Roncesvalles",Gift Shop,Dessert Shop,Coffee Shop,Movie Theater,Dog Run,Eastern European Restaurant,Italian Restaurant,Restaurant,Cuban Restaurant,Dance Studio


# Cluster 3

In [79]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Regent Park, Harbourfront",Distribution Center,Gym / Fitness Center,Bakery,Pub,Historic Site,Spa,Breakfast Spot,Restaurant,Park,Coffee Shop
2,"Garden District, Ryerson",Clothing Store,Burrito Place,Ramen Restaurant,Plaza,Pizza Place,Comic Shop,Tea Room,Thai Restaurant,Café,Music Venue
4,The Beaches,Trail,Asian Restaurant,Health Food Store,Pub,Wine Bar,Cuban Restaurant,Dessert Shop,Department Store,Dance Studio,Creperie
5,Berczy Park,Museum,Concert Hall,Restaurant,Breakfast Spot,Farmers Market,Liquor Store,Park,French Restaurant,Vegetarian / Vegan Restaurant,Cocktail Bar
8,"Richmond, Adelaide, King",Steakhouse,Restaurant,Plaza,Gym / Fitness Center,Speakeasy,Pizza Place,Hotel,Concert Hall,Vegetarian / Vegan Restaurant,Convenience Store
10,"Harbourfront East, Union Station, Toronto Islands",Salad Place,Hotel,Dessert Shop,Park,Performing Arts Venue,Lake,Skating Rink,Plaza,Sporting Goods Shop,Cosmetics Shop
11,"Little Portugal, Trinity",Wine Bar,Art Gallery,Pizza Place,Cuban Restaurant,Korean Restaurant,Brewery,Ice Cream Shop,Cocktail Bar,Asian Restaurant,New American Restaurant
15,"India Bazaar, The Beaches West",Fish & Chips Shop,Sushi Restaurant,Pub,Brewery,Gym,Fast Food Restaurant,Italian Restaurant,Ice Cream Shop,Liquor Store,Park
18,Lawrence Park,Park,Bus Line,Swim School,Lawyer,Wine Bar,Diner,Coffee Shop,Comic Shop,Concert Hall,Convenience Store
20,Davisville North,Gym,Convenience Store,Department Store,Hotel,Breakfast Spot,Pizza Place,Sandwich Place,Food & Drink Shop,Park,Concert Hall


# Cluster 4

In [80]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 3, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
29,"Moore Park, Summerhill East",Park,Playground,Restaurant,Wine Bar,Chinese Restaurant,Cocktail Bar,Coffee Shop,Comic Shop,Concert Hall,Convenience Store
33,Rosedale,Park,Trail,Playground,Wine Bar,Diner,Cocktail Bar,Coffee Shop,Comic Shop,Concert Hall,Convenience Store


# Cluster 5

In [81]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 4, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
19,Roselawn,Garden,Garden Center,Cocktail Bar,Coffee Shop,Comic Shop,Concert Hall,Convenience Store,Cosmetics Shop,Creperie,Cuban Restaurant
